In [1]:
from sklearn.pipeline import Pipeline
import pandas as pd
import category_encoders as ce
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score


eu_red_list = pd.read_csv("/Users/kristianjespersen/Documents/IKT/6_semester/ITMAL/itmal_4_redlist/datasets_ready/eu_red_list_train.csv")



encoder_pipeline = Pipeline([
  ('featureEncoding', ce.OneHotEncoder(cols=['speciesGroup','taxonomicRankKingdom','taxonomicRankPhylum','taxonomicRankClass','taxonomicRankOrder','taxonomicRankFamily','taxonomicRankGenus','taxonomicRankSpecies','endemicToEurope','populationTrend','useAndTrade'], return_df=True)),
  ('targetEncoding', ce.OrdinalEncoder(cols=['europeanRegionalRedListCategory'], return_df=True))
])

encoded = encoder_pipeline.fit_transform(eu_red_list)
X = encoded.iloc[:,:-1]
y = encoded.iloc[:,-1]
print(cross_val_score(MLPClassifier(),X,y,cv=5))

ModuleNotFoundError: No module named 'category_encoders'